In [1]:
# ============================================================================
# API DATA INGESTION - BRONZE LAYER
# ============================================================================

import requests
import json
from datetime import datetime, timedelta
from pyspark.sql.functions import col, lit, current_timestamp, explode, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, MapType

print("="*80)
print("🌐 API DATA INGESTION BAŞLIYOR")
print("="*80)
print(f"⏰ Çalışma zamanı: {datetime.now()}")

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 3, Finished, Available, Finished)

🌐 API DATA INGESTION BAŞLIYOR
⏰ Çalışma zamanı: 2025-11-29 16:10:10.916927


In [2]:
# Tüm tabloları listele
print("📋 Mevcut Tablolar:")
spark.sql("SHOW TABLES").show(truncate=False)

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 4, Finished, Available, Finished)

📋 Mevcut Tablolar:
+----------------+----------------------------+-----------+
|namespace       |tableName                   |isTemporary|
+----------------+----------------------------+-----------+
|bronze_lakehouse|bronze_hotel_bookings       |false      |
|bronze_lakehouse|bronze_hotel_bookings_stream|false      |
|bronze_lakehouse|bronze_weather_data         |false      |
|bronze_lakehouse|bronze_currency_rates       |false      |
|bronze_lakehouse|silver_hotel_bookings       |false      |
|bronze_lakehouse|silver_weather_data         |false      |
|bronze_lakehouse|silver_currency_rates       |false      |
|bronze_lakehouse|data_quality_reports        |false      |
|bronze_lakehouse|gold_dim_hotel              |false      |
|bronze_lakehouse|gold_dim_customer           |false      |
|bronze_lakehouse|gold_dim_city               |false      |
|bronze_lakehouse|gold_fact_booking           |false      |
|bronze_lakehouse|gold_kpi_revenue            |false      |
|bronze_lakehouse|gol

In [3]:
# ============================================================================
# TABLO KONTROLÜ
# ============================================================================

print("🔍 Lakehouse Durumu Kontrolü")
print("="*80)

# 1. Tüm tabloları listele
print("\n1️⃣ Tüm Tablolar:")
all_tables = spark.sql("SHOW TABLES")
all_tables.show(truncate=False)

# 2. Bronze tablolarını filtrele
print("\n2️⃣ Bronze Tabloları:")
bronze_tables = all_tables.filter(all_tables.tableName.startswith("bronze"))
bronze_tables.show(truncate=False)

bronze_count = bronze_tables.count()
print(f"\n✅ Toplam {bronze_count} bronze tablosu bulundu")

# 3. Eğer tablo varsa test et
if bronze_count > 0:
    table_name = bronze_tables.first()['tableName']
    print(f"\n3️⃣ Test: {table_name} tablosu okunuyor...")
    df_test = spark.table(table_name)
    print(f"✅ Başarılı! Satır sayısı: {df_test.count():,}")
else:
    print("\n⚠️ Hiç bronze tablosu bulunamadı!")
    print("💡 Önce ADIM 1 notebook'unu çalıştırmanız gerekiyor")

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 5, Finished, Available, Finished)

🔍 Lakehouse Durumu Kontrolü

1️⃣ Tüm Tablolar:
+----------------+----------------------------+-----------+
|namespace       |tableName                   |isTemporary|
+----------------+----------------------------+-----------+
|bronze_lakehouse|bronze_hotel_bookings       |false      |
|bronze_lakehouse|bronze_hotel_bookings_stream|false      |
|bronze_lakehouse|bronze_weather_data         |false      |
|bronze_lakehouse|bronze_currency_rates       |false      |
|bronze_lakehouse|silver_hotel_bookings       |false      |
|bronze_lakehouse|silver_weather_data         |false      |
|bronze_lakehouse|silver_currency_rates       |false      |
|bronze_lakehouse|data_quality_reports        |false      |
|bronze_lakehouse|gold_dim_hotel              |false      |
|bronze_lakehouse|gold_dim_customer           |false      |
|bronze_lakehouse|gold_dim_city               |false      |
|bronze_lakehouse|gold_fact_booking           |false      |
|bronze_lakehouse|gold_kpi_revenue            |false 

In [4]:
# ============================================================================
# ŞEHİR LİSTESİ - Batch/Stream verilerinden unique şehirler
# ============================================================================

print("\n" + "="*80)
print("🏙️ ŞEHİR LİSTESİ HAZIRLANIYOR")
print("="*80)

# Batch tablosundan şehirler (namespace ile)
df_batch = spark.table("bronze_lakehouse.bronze_hotel_bookings")
cities_batch = df_batch.select("city", "country").distinct()

print(f"✅ Batch'ten {cities_batch.count()} unique şehir bulundu")

# İlk 10 şehir
print("\n📍 Örnek şehirler:")
cities_batch.show(10, truncate=False)

# Pandas'a çevirip liste haline getir (API çağrıları için)
cities_list = cities_batch.toPandas().to_dict('records')

print(f"\n✅ API için {len(cities_list)} şehir hazırlandı")

# İlk 20 şehirle sınırlayalım (demo için - tümü çok uzun sürer)
cities_sample = cities_list[:20]
print(f"🎯 İlk {len(cities_sample)} şehir için API çağrısı yapılacak")

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 6, Finished, Available, Finished)


🏙️ ŞEHİR LİSTESİ HAZIRLANIYOR
✅ Batch'ten 61760 unique şehir bulundu

📍 Örnek şehirler:
+-----------+-----------+
|city       |country    |
+-----------+-----------+
|Toulouse   |France     |
|-142.115307|Ten        |
|GUADALAJARA|Mexico     |
|Dubrovnik  |Croatia    |
|-2.395627  |-76.823159 |
|!!False    |40         |
|Sydney     |!!Australia|
|___        |Mexico...  |
|165.678214 |30.13229   |
|BERGEN     |Norway     |
+-----------+-----------+
only showing top 10 rows


✅ API için 61760 şehir hazırlandı
🎯 İlk 20 şehir için API çağrısı yapılacak


In [5]:
# ============================================================================
# WEATHER API - GEOCODİNG (Şehir → Koordinat)
# ============================================================================
print("\n" + "="*80)
print("🗺️ GEOCODİNG - Şehir Koordinatları Alınıyor")
print("="*80)

def get_coordinates(city, country):
    """Şehir ve ülke için koordinat al"""
    try:
        # Open-Meteo Geocoding API
        url = "https://geocoding-api.open-meteo.com/v1/search"
        params = {
            "name": city,
            "count": 1,
            "language": "en",
            "format": "json"
        }
        
        response = requests.get(url, params=params, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            if "results" in data and len(data["results"]) > 0:
                result = data["results"][0]
                return {
                    "city": city,
                    "country": country,
                    "latitude": result.get("latitude"),
                    "longitude": result.get("longitude"),
                    "timezone": result.get("timezone", "UTC")
                }
    except Exception as e:
        print(f"❌ Hata ({city}): {str(e)}")
    
    return None

# Koordinatları al
coordinates = []
successful = 0
failed = 0

print(f"🔄 {len(cities_sample)} şehir için geocoding başlıyor...\n")

for i, city_info in enumerate(cities_sample, 1):
    city = city_info.get('city', '')
    country = city_info.get('country', '')
    
    # Geçersiz değerleri atla
    invalid_values = ['--', '???', '___', 'NULL', '', 'Ten', '!!False']
    
    # Şehir isminin sayı olup olmadığını kontrol et
    try:
        float(city)
        continue  # Sayıysa atla
    except (ValueError, TypeError):
        pass  # Sayı değilse devam et
    
    if city and city not in invalid_values and len(city) > 2:
        coord = get_coordinates(city, country)
        if coord:
            coordinates.append(coord)
            successful += 1
            if successful % 5 == 0:  # Her 5'te bir progress
                print(f"✅ {successful} şehir başarılı, {failed} başarısız")
        else:
            failed += 1

print(f"\n" + "="*50)
print(f"✅ Toplam başarılı: {successful}")
print(f"❌ Toplam başarısız: {failed}")
print(f"📍 Koordinat alınan şehir sayısı: {len(coordinates)}")
print("="*50)

# Örnek koordinatlar
if coordinates:
    import pandas as pd
    print("\n📊 İlk 5 Şehir Koordinatları:")
    print(pd.DataFrame(coordinates[:5]))

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 7, Finished, Available, Finished)


🗺️ GEOCODİNG - Şehir Koordinatları Alınıyor
🔄 20 şehir için geocoding başlıyor...

✅ 5 şehir başarılı, 0 başarısız

✅ Toplam başarılı: 9
❌ Toplam başarısız: 1
📍 Koordinat alınan şehir sayısı: 9

📊 İlk 5 Şehir Koordinatları:
          city      country  latitude  longitude             timezone
0     Toulouse       France  43.60426    1.44367         Europe/Paris
1  GUADALAJARA       Mexico  20.66682 -103.39182  America/Mexico_City
2    Dubrovnik      Croatia  42.64125   18.10909        Europe/Zagreb
3       Sydney  !!Australia -33.86785  151.20732     Australia/Sydney
4       BERGEN       Norway  60.39299    5.32415          Europe/Oslo


In [6]:
# ============================================================================
# WEATHER API - Hava Durumu Verisi
# ============================================================================

print("\n" + "="*80)
print("HAVA DURUMU VERİSİ ALINIYOR")
print("="*80)

def get_weather(latitude, longitude, city):
    """Koordinatlar için hava durumu al"""
    try:
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current": "temperature_2m,relative_humidity_2m,wind_speed_10m,weather_code",
            "timezone": "auto"
        }
        
        response = requests.get(url, params=params, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            current = data.get("current", {})
            
            return {
                "city": city,
                "latitude": latitude,
                "longitude": longitude,
                "temperature_celsius": current.get("temperature_2m"),
                "humidity_percent": current.get("relative_humidity_2m"),
                "wind_speed_kmh": current.get("wind_speed_10m"),
                "weather_code": current.get("weather_code"),
                "timestamp": current.get("time"),
                "api_call_time": datetime.utcnow().isoformat()
            }
    except Exception as e:
        print(f"Hata ({city}): {str(e)}")
    
    return None

# Hava durumu verilerini al
weather_data = []
successful = 0
failed = 0

print(f"\n{len(coordinates)} sehir icin hava durumu aliniyor...\n")

for i, coord in enumerate(coordinates, 1):
    weather = get_weather(
        coord['latitude'], 
        coord['longitude'], 
        coord['city']
    )
    
    if weather:
        weather_data.append(weather)
        successful += 1
        if successful % 3 == 0:
            print(f">>> {successful}/{len(coordinates)} sehir tamamlandi")
    else:
        failed += 1

print("\n" + "="*50)
print(f"Basarili: {successful}")
print(f"Basarisiz: {failed}")
print(f"Hava durumu verisi: {len(weather_data)} sehir")
print("="*50)

# Örnek veri
if weather_data:
    import pandas as pd
    print("\nOrnek Hava Durumu Verileri:")
    df_weather_preview = pd.DataFrame(weather_data[:5])
    print(df_weather_preview.to_string(index=False))

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 8, Finished, Available, Finished)


HAVA DURUMU VERİSİ ALINIYOR

9 sehir icin hava durumu aliniyor...

>>> 3/9 sehir tamamlandi
>>> 6/9 sehir tamamlandi
>>> 9/9 sehir tamamlandi

Basarili: 9
Basarisiz: 0
Hava durumu verisi: 9 sehir

Ornek Hava Durumu Verileri:
       city  latitude  longitude  temperature_celsius  humidity_percent  wind_speed_kmh  weather_code        timestamp              api_call_time
   Toulouse  43.60426    1.44367                 11.2                73             2.3             3 2025-11-29T17:00 2025-11-29T16:10:50.580351
GUADALAJARA  20.66682 -103.39182                 19.9                53             2.2             3 2025-11-29T10:00 2025-11-29T16:10:50.757830
  Dubrovnik  42.64125   18.10909                 12.2                57             1.9             3 2025-11-29T17:00 2025-11-29T16:10:50.893266
     Sydney -33.86785  151.20732                 17.0                79             2.5             2 2025-11-30T03:00 2025-11-29T16:10:51.029202
     BERGEN  60.39299    5.32415            

In [7]:
# ============================================================================
# CURRENCY API - Doviz Kurlari
# ============================================================================

print("\n" + "="*80)
print("DOVIZ KURLARI ALINIYOR")
print("="*80)

def get_exchange_rates():
    """USD bazli doviz kurlarini al"""
    try:
        url = "https://open.er-api.com/v6/latest/USD"
        response = requests.get(url, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            
            return {
                "base_currency": data.get("base_code", "USD"),
                "last_updated": data.get("time_last_update_utc"),
                "rates": data.get("rates", {}),
                "api_call_time": datetime.utcnow().isoformat()
            }
    except Exception as e:
        print(f"Doviz API hatasi: {str(e)}")
    
    return None

# Doviz kurlarini al
currency_data = get_exchange_rates()

if currency_data:
    print("Doviz kurlari alindi!")
    print(f"Son guncelleme: {currency_data['last_updated']}")
    print(f"Base currency: {currency_data['base_currency']}")
    print(f"Toplam {len(currency_data['rates'])} doviz kuru mevcut")
    
    # Onemli kurlar
    important_currencies = ['EUR', 'GBP', 'JPY', 'TRY', 'CNY', 'AUD', 'CAD', 'CHF']
    print("\nOnemli Doviz Kurlari (USD bazli):")
    for curr in important_currencies:
        rate = currency_data['rates'].get(curr, 'N/A')
        print(f"  {curr}: {rate}")
else:
    print("Doviz kurlari alinamadi!")

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 9, Finished, Available, Finished)


DOVIZ KURLARI ALINIYOR
Doviz kurlari alindi!
Son guncelleme: Sat, 29 Nov 2025 00:02:32 +0000
Base currency: USD
Toplam 166 doviz kuru mevcut

Onemli Doviz Kurlari (USD bazli):
  EUR: 0.862834
  GBP: 0.755703
  JPY: 156.177017
  TRY: 42.501917
  CNY: 7.083189
  AUD: 1.528367
  CAD: 1.398495
  CHF: 0.804195


In [8]:
print("\n" + "="*80)
print("WEATHER VERISI BRONZE'A YAZILIYOR")
print("="*80)

if weather_data:
    # Pandas DataFrame'e cevir
    import pandas as pd
    df_weather_pd = pd.DataFrame(weather_data)
    
    # Spark DataFrame'e cevir
    df_weather = spark.createDataFrame(df_weather_pd)
    
    # Metadata ekle
    df_weather = df_weather.withColumn("ingestion_timestamp", current_timestamp()) \
                           .withColumn("source_api", lit("open-meteo")) \
                           .withColumn("layer", lit("bronze"))
    
    print(f"DataFrame hazir: {df_weather.count()} satir")
    
    # Delta formatinda yaz (namespace ile)
    df_weather.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("bronze_lakehouse.bronze_weather_data")
    
    print(f"Weather verisi Bronze'a yazildi!")
    print(f"Tablo: bronze_lakehouse.bronze_weather_data")
    print(f"Satir sayisi: {df_weather.count()}")
    
    # Onizleme
    print("\nIlk 5 satir:")
    df_weather.show(5, truncate=False)
else:
    print("Weather verisi yok, yazma atlandi")

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 10, Finished, Available, Finished)


WEATHER VERISI BRONZE'A YAZILIYOR
DataFrame hazir: 9 satir
Weather verisi Bronze'a yazildi!
Tablo: bronze_lakehouse.bronze_weather_data
Satir sayisi: 9

Ilk 5 satir:
+-----------+---------+----------+-------------------+----------------+--------------+------------+----------------+--------------------------+-------------------------+----------+------+
|city       |latitude |longitude |temperature_celsius|humidity_percent|wind_speed_kmh|weather_code|timestamp       |api_call_time             |ingestion_timestamp      |source_api|layer |
+-----------+---------+----------+-------------------+----------------+--------------+------------+----------------+--------------------------+-------------------------+----------+------+
|Toulouse   |43.60426 |1.44367   |11.2               |73              |2.3           |3           |2025-11-29T17:00|2025-11-29T16:10:50.580351|2025-11-29 16:11:08.91586|open-meteo|bronze|
|GUADALAJARA|20.66682 |-103.39182|19.9               |53              |2.2       

In [9]:
print("\n" + "="*80)
print("CURRENCY VERISI BRONZE'A YAZILIYOR")
print("="*80)

if currency_data:
    # Her doviz kuru icin ayri satir olustur
    currency_records = []
    for currency_code, rate in currency_data['rates'].items():
        currency_records.append({
            "base_currency": currency_data['base_currency'],
            "target_currency": currency_code,
            "exchange_rate": float(rate),
            "last_updated": currency_data['last_updated'],
            "api_call_time": currency_data['api_call_time']
        })
    
    print(f"{len(currency_records)} doviz kuru kaydi hazirlandi")
    
    # DataFrame olustur
    import pandas as pd
    df_currency_pd = pd.DataFrame(currency_records)
    df_currency = spark.createDataFrame(df_currency_pd)
    
    # Metadata ekle
    df_currency = df_currency.withColumn("ingestion_timestamp", current_timestamp()) \
                             .withColumn("source_api", lit("exchangerate-api")) \
                             .withColumn("layer", lit("bronze"))
    
    # Delta formatinda yaz (namespace ile)
    df_currency.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("bronze_lakehouse.bronze_currency_rates")
    
    print(f"Currency verisi Bronze'a yazildi!")
    print(f"Tablo: bronze_lakehouse.bronze_currency_rates")
    print(f"Satir sayisi: {df_currency.count()}")
    
    # Onizleme - onemli kurlar
    print("\nOnemli doviz kurlari (ilk 10):")
    df_currency.filter(
        col("target_currency").isin(['EUR', 'GBP', 'JPY', 'TRY', 'CNY', 'AUD', 'CAD', 'CHF', 'INR', 'BRL'])
    ).show(10, truncate=False)
else:
    print("Currency verisi yok, yazma atlandi")

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 11, Finished, Available, Finished)


CURRENCY VERISI BRONZE'A YAZILIYOR
166 doviz kuru kaydi hazirlandi
Currency verisi Bronze'a yazildi!
Tablo: bronze_lakehouse.bronze_currency_rates
Satir sayisi: 166

Onemli doviz kurlari (ilk 10):
+-------------+---------------+-------------+-------------------------------+--------------------------+--------------------------+----------------+------+
|base_currency|target_currency|exchange_rate|last_updated                   |api_call_time             |ingestion_timestamp       |source_api      |layer |
+-------------+---------------+-------------+-------------------------------+--------------------------+--------------------------+----------------+------+
|USD          |AUD            |1.528367     |Sat, 29 Nov 2025 00:02:32 +0000|2025-11-29T16:10:52.569861|2025-11-29 16:11:20.101553|exchangerate-api|bronze|
|USD          |BRL            |5.345969     |Sat, 29 Nov 2025 00:02:32 +0000|2025-11-29T16:10:52.569861|2025-11-29 16:11:20.101553|exchangerate-api|bronze|
|USD          |CAD    

In [10]:
# ============================================================================
# BRONZE LAYER OZET RAPOR
# ============================================================================

print("\n" + "="*80)
print("BRONZE LAYER - TUM TABLOLAR")
print("="*80)

# Tum tablolari listele
bronze_tables = spark.sql("SHOW TABLES IN bronze_lakehouse").filter(col("tableName").startswith("bronze_"))

print("\nBronze Layer Tablolari:")
bronze_tables.show(truncate=False)

# Her tablo icin satir sayisi
print("\n" + "="*80)
print("TABLO ISTATISTIKLERI")
print("="*80)

table_stats = []

for table_row in bronze_tables.collect():
    table_name = table_row['tableName']
    full_table_name = f"bronze_lakehouse.{table_name}"
    
    try:
        count = spark.table(full_table_name).count()
        table_stats.append({
            "table_name": table_name,
            "row_count": count
        })
        print(f"  {table_name}: {count:,} satir")
    except Exception as e:
        print(f"  {table_name}: HATA - {str(e)}")

print("\n" + "="*80)
print("ADIM 4 TAMAMLANDI!")
print("="*80)

# Ozet
total_rows = sum([s['row_count'] for s in table_stats])
print(f"\nBronze Layer Ozeti:")
print(f"  Toplam tablo sayisi: {len(table_stats)}")
print(f"  Toplam satir sayisi: {total_rows:,}")
print(f"\nTablo detaylari:")
for stat in table_stats:
    print(f"  - {stat['table_name']}: {stat['row_count']:,} satir")

print("\n" + "="*80)
print("Sirada: ADIM 5 - Silver Layer Transformations")
print("="*80)

StatementMeta(, 93af61a9-1958-4289-b77e-f8736d419f68, 12, Finished, Available, Finished)


BRONZE LAYER - TUM TABLOLAR

Bronze Layer Tablolari:
+----------------+----------------------------+-----------+
|namespace       |tableName                   |isTemporary|
+----------------+----------------------------+-----------+
|bronze_lakehouse|bronze_hotel_bookings       |false      |
|bronze_lakehouse|bronze_hotel_bookings_stream|false      |
|bronze_lakehouse|bronze_weather_data         |false      |
|bronze_lakehouse|bronze_currency_rates       |false      |
+----------------+----------------------------+-----------+


TABLO ISTATISTIKLERI
  bronze_hotel_bookings: 1,363,403 satir
  bronze_hotel_bookings_stream: 450,000 satir
  bronze_weather_data: 9 satir
  bronze_currency_rates: 166 satir

ADIM 4 TAMAMLANDI!

Bronze Layer Ozeti:
  Toplam tablo sayisi: 4
  Toplam satir sayisi: 1,813,578

Tablo detaylari:
  - bronze_hotel_bookings: 1,363,403 satir
  - bronze_hotel_bookings_stream: 450,000 satir
  - bronze_weather_data: 9 satir
  - bronze_currency_rates: 166 satir

Sirada: ADI